# Preparation

## import tools

In [1]:
import os, numpy as np
import histogram.hdf as hh
import mcvine, mcvine.components

from matplotlib import pyplot as plt
%matplotlib notebook

## work dir

In [2]:
workdir = os.path.expanduser("~/simulations/mcvine/demo/python-based-neutron-component")
!mkdir -p {workdir}
%cd {workdir}

/SNS/users/lj7/simulations/mcvine/demo/python-based-neutron-component


# Custom component

In [37]:
from mcni.AbstractComponent import AbstractComponent
import numpy as np

class EventMonitor_4D( AbstractComponent ):

    def __init__(self, name, xwidth, yheight):
        self.name = name
        self.xwidth= xwidth; self.yheight = yheight
        return
    
    def process(self, neutrons):
        if not len(neutrons):
            return
        from mcni.neutron_storage import neutrons_as_npyarr, ndblsperneutron
        arr = neutrons_as_npyarr(neutrons)
        arr.shape = -1, ndblsperneutron
        x = arr[:,0]; y = arr[:,1]; z = arr[:,2]
        vx = arr[:,3]; vy = arr[:,4]; vz = arr[:,5]
        s1 = arr[:,6]; s2 = arr[:,7];
        t = arr[:,8]; 
        p = arr[:,9]

        # propagate to z = 0
        self._propagateToZ0(x,y,z,vx,vy,vz,t)

        # Apply filter if area is positive
        assert self.xwidth > 0 and self.yheight > 0

        # Filter
        ftr = (x >= -self.xwidth/2)*(x <= self.xwidth/2)*(y >= -self.yheight/2)*(y <= self.yheight/2)
        x = x[ftr]; y = y[ftr]; z = z[ftr];
        vx = vx[ftr]; vy = vy[ftr]; vz = vz[ftr];
        s1 = s1[ftr]; s2 = s2[ftr]; t = t[ftr]; p = p[ftr];
        events = x,y,z,t
        self.save(events)
        return
    
    def save(self, events):
        outdir = self.simulation_context.getOutputDirInProgress()
        np.save(os.path.join(outdir, "events4D.npy"), events)
    
    def _propagateToZ0(self, x,y,z, vx,vy,vz, t):
        dt = -z/vz
        x += vx*dt
        y += vy*dt
        z[:] = 0
        t += dt
        return

out-mcvine/step0


## Test it wit a simple instrument

In [39]:
instrument = mcvine.instrument()
instrument.append(mcvine.components.sources.Source_simple('source'), position=(0,0,0))
instrument.append(EventMonitor_4D('monitor', 0.1, 0.1), position=(0,0,1))

### run simulation

In [41]:
%%time
# quick sim
neutrons = instrument.simulate(int(10),outputdir="out-test0", overwrite_datafiles=True, iteration_no=0)

CPU times: user 8.78 ms, sys: 586 µs, total: 9.37 ms
Wall time: 28.3 ms


### check simulation results

In [44]:
for i in range(5):
    print neutrons[i]

Neutron( state=NeutronState( position=(0.0098459,0.0160985,0), velocity=(-5.8209,-10.6702,3516.34), spin=(0, 0) ), time=0, probability=0.00785395 )
Neutron( state=NeutronState( position=(-0.0139586,0.0320981,0), velocity=(-6.69708,-11.0707,3409.66), spin=(0, 0) ), time=0, probability=0.00785385 )
Neutron( state=NeutronState( position=(0.0413984,-0.00320358,0), velocity=(-0.495277,-7.04656,3429.9), spin=(0, 0) ), time=0, probability=0.00785373 )
Neutron( state=NeutronState( position=(-0.00313726,0.039664,0), velocity=(15.9376,-17.141,3485.57), spin=(0, 0) ), time=0, probability=0.00785376 )
Neutron( state=NeutronState( position=(-0.0305301,-0.0275972,0), velocity=(10.797,18.0873,3281.71), spin=(0, 0) ), time=0, probability=0.00785389 )


## Test parallel simulation of the new component

### create script

In [60]:
%%file myinstrument.py
import mcvine, mcvine.components
from mcni.AbstractComponent import AbstractComponent
import numpy as np
import os

class EventMonitor_4D( AbstractComponent ):

    def __init__(self, name, xwidth, yheight):
        self.name = name
        self.xwidth= xwidth; self.yheight = yheight
        return
    
    def process(self, neutrons):
        if not len(neutrons):
            return
        from mcni.neutron_storage import neutrons_as_npyarr, ndblsperneutron
        arr = neutrons_as_npyarr(neutrons)
        arr.shape = -1, ndblsperneutron
        x = arr[:,0]; y = arr[:,1]; z = arr[:,2]
        vx = arr[:,3]; vy = arr[:,4]; vz = arr[:,5]
        s1 = arr[:,6]; s2 = arr[:,7];
        t = arr[:,8]; 
        p = arr[:,9]

        # propagate to z = 0
        self._propagateToZ0(x,y,z,vx,vy,vz,t)

        # Apply filter if area is positive
        assert self.xwidth > 0 and self.yheight > 0

        # Filter
        ftr    = (x >= -self.xwidth/2)*(x <= self.xwidth/2)*(y >= -self.yheight/2)*(y <= self.yheight/2)

        x = x[ftr]; y = y[ftr]; z = z[ftr];
        vx = vx[ftr]; vy = vy[ftr]; vz = vz[ftr];
        s1 = s1[ftr]; s2 = s2[ftr]; t = t[ftr]; p = p[ftr];
        events = x,y,z,t
        self.save(events)
        return
    
    def save(self, events):
        outdir = self._getOutputDirInProgress()
        print outdir
        np.save(os.path.join(outdir, "events4D.npy"), events)
    
    def _propagateToZ0(self, x,y,z, vx,vy,vz, t):
        dt = -z/vz
        x += vx*dt
        y += vy*dt
        z[:] = 0
        t += dt
        return
    
instrument = mcvine.instrument()
instrument.append(mcvine.components.sources.Source_simple('source'), position=(0,0,0))
instrument.append(EventMonitor_4D('monitor', 0.1, 0.1), position=(0,0,1))

Writing myinstrument.py


### run script

In [50]:
from mcvine import run_script

In [61]:
%%time
run_script.run_mpi('./myinstrument.py', 'out-test-parallel', ncount=1e3, nodes=10, overwrite_datafiles=True)

CPU times: user 1.69 ms, sys: 3.34 ms, total: 5.03 ms
Wall time: 4.51 s


### check results

In [62]:
ls out-test-parallel/

post-processing-scripts/  rank2-step2/  rank5-step0/  rank7-step3/
rank0-step0/              rank2-step3/  rank5-step1/  rank7-step4/
rank0-step1/              rank2-step4/  rank5-step2/  rank8-step0/
rank0-step2/              rank3-step0/  rank5-step3/  rank8-step1/
rank0-step3/              rank3-step1/  rank5-step4/  rank8-step2/
rank0-step4/              rank3-step2/  rank6-step0/  rank8-step3/
rank1-step0/              rank3-step3/  rank6-step1/  rank8-step4/
rank1-step1/              rank3-step4/  rank6-step2/  rank9-step0/
rank1-step2/              rank4-step0/  rank6-step3/  rank9-step1/
rank1-step3/              rank4-step1/  rank6-step4/  rank9-step2/
rank1-step4/              rank4-step2/  rank7-step0/  rank9-step3/
rank2-step0/              rank4-step3/  rank7-step1/  rank9-step4/
rank2-step1/              rank4-step4/  rank7-step2/


In [63]:
# !find . -name *.npy

In [65]:
cat out-test-parallel/rank0-step0/number_of_mc_samples

20

In [66]:
ls out-test-parallel/rank0-step0/

events4D.npy  number_of_mc_samples


In [67]:
events = np.load("out-test-parallel/rank0-step0/events4D.npy")

In [68]:
events

array([[-0.01514469,  0.02710665, -0.01122233,  0.01765849, -0.0403612 ,
         0.01716935, -0.03839571, -0.0353358 ,  0.04145368, -0.03400628,
        -0.00047947, -0.02838686,  0.02717772, -0.01705978,  0.00775866,
         0.01735421, -0.00607266, -0.02794056, -0.01268873, -0.03678636],
       [ 0.03180821, -0.00954576, -0.03176664, -0.01579207,  0.00037146,
         0.03436178, -0.01608668, -0.01999175, -0.02617493,  0.01938692,
         0.0283458 ,  0.02041745, -0.02536763,  0.01417415, -0.01943654,
        -0.01654378, -0.00261504,  0.02837345, -0.01922561,  0.00244225],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.00029569,  0.00031024,  0.00029495,  0.00029686,  0.00030782,
         0.00027662,  0.00030546,  0.00030424,  